y = ReLU(w * x + b)

Active set (where w * x + b > 0) iterative fitting

In [28]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load and prepare the California Housing dataset
data = fetch_california_housing()
X, y = data.data, data.target

# Standardize features and reshape target to a column vector
scaler = StandardScaler()
X = scaler.fit_transform(X)
y = y.reshape(-1, 1)

# Add a column of ones to X to include the bias term
X = np.hstack([X, np.ones((X.shape[0], 1))])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

np.random.seed(42)
# Random initialization of weights, scaled down to avoid large initial values
w = np.random.randn(X_train.shape[1], 1) * 0.01  # Adjusted for bias term

max_iterations = 100
previous_S = None

for iteration in range(max_iterations):
    z = X_train @ w
    S = z.flatten() > 0  # Active set where z_i > 0

    # Check for convergence
    if previous_S is not None and np.array_equal(S, previous_S):
        print(f"Converged after {iteration} iterations.")
        break
    previous_S = S.copy()

    # Subset matrices and vectors based on active set S
    X_S = X_train[S]
    y_S = y_train[S]

    # Solve for w using the closed-form solution
    w = np.linalg.pinv(X_S.T @ X_S) @ X_S.T @ y_S
    
    print(f"Iteration {iteration + 1}, Active set size: {np.sum(S)}/{len(X_train)}")

else:
    print("Maximum iterations reached without convergence.")

# Compute final SSE on the test set
z_test = X_test @ w
y_pred_test = np.maximum(0, z_test)
SSE_test = np.sum((y_test - y_pred_test) ** 2)
print("Final MSE on test set:", SSE_test / len(y_test))

# Compute final SSE on the training set
z_train = X_train @ w
y_pred_train = np.maximum(0, z_train)
SSE_train = np.sum((y_train - y_pred_train) ** 2)
print("Final MSE on train set:", SSE_train / len(y_train))


Iteration 1, Active set size: 5035/16512
Iteration 2, Active set size: 16464/16512
Iteration 3, Active set size: 16399/16512
Iteration 4, Active set size: 16373/16512
Iteration 5, Active set size: 16368/16512
Converged after 5 iterations.
Final MSE on test set: 0.4827624956396167
Final MSE on train set: 0.45740270445540165


In [29]:
w

array([[ 0.79207735],
       [ 0.13726904],
       [-0.18946688],
       [ 0.25255451],
       [ 0.04693232],
       [-3.79466347],
       [-0.99214778],
       [-0.90929532],
       [ 2.00451055]])

In [31]:
w

array([[ 0.79604198],
       [ 0.13599718],
       [-0.19746062],
       [ 0.2609147 ],
       [ 0.04340834],
       [-3.54717451],
       [-0.98981705],
       [-0.91022543]])

Comparing to gradient descent

In [30]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load and prepare the California Housing dataset
data = fetch_california_housing()
X, y = data.data, data.target

# Standardize features and reshape target to a column vector
scaler = StandardScaler()
X = scaler.fit_transform(X)
y = y.reshape(-1, 1)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Initialize weights and bias
D = X_train.shape[1]

np.random.seed(42)
w = np.random.randn(D, 1) * 0.01
b = np.random.randn(1) * 0.01  # Initialize bias

# Hyperparameters
learning_rate = 0.00001
num_epochs = 2000

# Training loop
for epoch in range(num_epochs):
    # Forward pass
    Xw = X_train @ w + b  # Include bias term
    y_pred = np.maximum(0, Xw)  # ReLU activation

    # Compute loss
    loss = np.sum((y_train - y_pred) ** 2)

    # Compute gradient
    relu_grad = (Xw > 0).astype(float)  # Derivative of ReLU
    error = y_pred - y_train
    grad_w = 2 * X_train.T @ (relu_grad * error)
    grad_b = 2 * np.sum(relu_grad * error)

    # Update weights and bias
    w -= learning_rate * grad_w
    b -= learning_rate * grad_b

    # Optionally print loss every 100 epochs
    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss}")

# After training, w and b contain the optimized weights and bias
# print("Optimized weights:\n", w)
# print("Optimized bias:\n", b)

# Compute SSE on the test set
Xw_test = X_test @ w + b
y_test_pred = np.maximum(0, Xw_test)
SSE_test = np.sum((y_test - y_test_pred) ** 2)
print("Final MSE on test set:", SSE_test / len(y_test))

# Compute SSE on the training set
Xw_train = X_train @ w + b
y_pred_train = np.maximum(0, Xw_train)
SSE_train = np.sum((y_train - y_pred_train) ** 2)
print("Final MSE on train set:", SSE_train / len(y_train))


Epoch 50/2000, Loss: 8688.868823921486
Epoch 100/2000, Loss: 8343.769116423062
Epoch 150/2000, Loss: 8222.973305135594
Epoch 200/2000, Loss: 8141.520770507027
Epoch 250/2000, Loss: 8075.374669593973
Epoch 300/2000, Loss: 8020.006917634892
Epoch 350/2000, Loss: 7965.582213307688
Epoch 400/2000, Loss: 7911.849656344966
Epoch 450/2000, Loss: 7863.029718076686
Epoch 500/2000, Loss: 7821.206848639167
Epoch 550/2000, Loss: 7785.630605651713
Epoch 600/2000, Loss: 7754.931054252702
Epoch 650/2000, Loss: 7728.262758130516
Epoch 700/2000, Loss: 7705.418210581371
Epoch 750/2000, Loss: 7685.980247568649
Epoch 800/2000, Loss: 7669.231109098092
Epoch 850/2000, Loss: 7653.282672283955
Epoch 900/2000, Loss: 7639.613421946356
Epoch 950/2000, Loss: 7627.872472542103
Epoch 1000/2000, Loss: 7617.733406607577
Epoch 1050/2000, Loss: 7609.082334531049
Epoch 1100/2000, Loss: 7601.693420503311
Epoch 1150/2000, Loss: 7595.308404373517
Epoch 1200/2000, Loss: 7589.689095780285
Epoch 1250/2000, Loss: 7584.91482479